In [46]:
import os
import re

from transformers import pipeline
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

import spacy

import pandas as pd

import math

extract name

In [47]:
#names
#names count

def extract_name(text):
  name_ls = []
  names_count = 0
  matches = re.findall(r'my name is (\w+) (\w+)', text, re.IGNORECASE)
  for match in matches:
    name_ls.append(match)
    names_count += 1
    
  return name_ls, names_count

extract keywords

In [48]:
#parking
#affordability
#community
#total words

def extract_keywords(text):
  words = text.split()
  words_count = len(words)

  word_hash = {}
  for word in words:
    if word not in word_hash:
        word_hash[word] = 1
    else:
        word_hash[word] += 1

  parking_count = 0
  if "parking" in word_hash:
     parking_count = word_hash["parking"]
  if "Parking" in word_hash:
     parking_count = word_hash["Parking"]
 

  affordable_count = 0
  if "affordability" in word_hash:
     affordable_count = word_hash["affordability"]
  if "affordable" in word_hash:
     affordable_count += word_hash["affordable"]
  if "Affordability" in word_hash:
     affordable_count = word_hash["Affordability"]
  if "Affordable" in word_hash:
     affordable_count += word_hash["Affordable"]
     

  community_count = 0
  if "community" in word_hash:
     community_count = word_hash["community"]
  if "communities" in word_hash:
     community_count += word_hash["communities"]
  if "Community" in word_hash:
     community_count = word_hash["Community"]
  if "Communities" in word_hash:
     community_count += word_hash["Communities"]



  
  return words_count, parking_count, affordable_count, community_count

sentiment

In [49]:
#sentiment
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"


tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def get_sentiment(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  # print(encoded_input['input_ids'].size())
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  return scores[0], scores[1], scores[2]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


word count

In [50]:
#text minus stop words
#text minus stop word count 

def remove_stop_words(text):
  words = text.split()
  non_stop_words_count = 0
  nlp = spacy.load("en_core_web_sm")
  stop_words = spacy.lang.en.stop_words.STOP_WORDS
  for i in range(len(words)):
     if words[i] not in stop_words:
        non_stop_words_count += 1
  return non_stop_words_count

merge

In [51]:
cleaned_hyperlinks_df = pd.read_csv("cleaned_hyperlinks.csv")
case_to_df = {}

collapsed_df = cleaned_hyperlinks_df.groupby(['clean_file_name', 'case_name'])['audio_file_name'].agg(list).reset_index()


df = pd.DataFrame(columns=['case_name', 'CEQA', 'names', 'names_count', 'parking_mentions', 'affordability_mentions', 'community_mentions', 'num_words', 'num_nonstopgap_words', 'sentiment_pos', 'sentiment_neg', 'sentiment_neutral', 'date', 'area', 'txt_file_name'])

In [52]:
collapsed_df

,clean_file_name,case_name,audio_file_name
0,01-05-10audio,01DepartmentalRepor,[01-05-10audio_01DepartmentalReport]
1,01-05-10audio,02CommissionBusines,[01-05-10audio_02CommissionBusiness]
2,01-05-10audio,03ZA09-2885&04ZA09-2886,"[01-05-10audio_03ZA09-2885_04ZA09-2886a, 01-05..."
3,01-05-10audio,04ZA09-2886,[01-05-10audio_04ZA09-2886]
4,01-05-10audio,05APCW10-657,[01-05-10audio_05APCW10-657]
...,...,...,...
2677,WLA12-05-12audio,04PublicCommentPerio,[WLA12-05-12audio_04PublicCommentPeriod]
2678,WLA12-07-11audio,01DepartmentalRepor,[WLA12-07-11audio_01DepartmentalReport]
2679,WLA12-07-11audio,02CommissionBusines,[WLA12-07-11audio_02CommissionBusiness]
2680,WLA12-07-11audio,03ZA11-1481,[WLA12-07-11audio_03ZA11-1481]


In [53]:
# filtered_rows = cleaned_hyperlinks_df[(cleaned_hyperlinks_df['clean_file_name'] == "07-12-2012audio") & (cleaned_hyperlinks_df['case_name'] == "01DirectorsRepor")]

# date = filtered_rows.loc[0, 'hearing_date']
# area = filtered_rows.loc[0, 'area']
# area
# date

In [54]:
#merge txt
#load in df
def get_date_location(clean_file_name, case_name):

  filtered_rows = cleaned_hyperlinks_df[(cleaned_hyperlinks_df['clean_file_name'] == clean_file_name) & (cleaned_hyperlinks_df['case_name'] == case_name)]

  date = filtered_rows.iloc[0]['hearing_date']
  area = filtered_rows.iloc[0]['area']

  return date, area


def get_ceqa(clean_file_name, case_name):
  filtered_rows = cleaned_hyperlinks_df[(cleaned_hyperlinks_df['clean_file_name'] == clean_file_name) & (cleaned_hyperlinks_df['case_name'] == case_name)]
  text_vals = filtered_rows['Text'].tolist()

  pattern = r"CEQA:\s*([\w-]+)"

  for text in text_vals:
    match = re.search(pattern, str(text))

    if match:
      code = match.group(1)
      return code
 
  
  return np.NaN

def split_text(text, chunk_size=1500):
    # words = text.split()  # Split the text into individual words
    # chunks = []
    # current_chunk = []  # Initialize an empty list to hold the words in the current chunk
    # word_count = 0  # Initialize a counter for the number of words in the current chunk
    
    # # Iterate over the words in the text
    # for word in words:
    #     current_chunk.append(word)  # Add the word to the current chunk
    #     word_count += 1  # Increment the word count
        
    #     # If the word count reaches the chunk size or if it's the last word
    #     if word_count == chunk_size or word == words[-1]:
    #         # Add the current chunk to the list of chunks
    #         chunks.append(' '.join(current_chunk))
    #         current_chunk = []  # Reset the current chunk
    #         word_count = 0  # Reset the word count
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        chunks.append(chunk)
    
    return chunks


for index, row in collapsed_df.iterrows():
  file_names = row['audio_file_name']
  clean_file_name = row['clean_file_name']
  case_name = row['case_name']

  ceqa = get_ceqa(clean_file_name, case_name)
  date, area = get_date_location(clean_file_name, case_name)
  text = " "
  for file in file_names:
    file_name = 'txt/' + file + '.txt'
    if os.path.exists(file_name):
      with open(file_name, 'r') as file:
        this_file_text = file.read()
        text += this_file_text
  print(file_name)
  name_ls, names_count = extract_name(text)
  words_count, parking_count, affordable_count, community_count = extract_keywords(text)
  # if words_count > 300:
  #   length = len(text)
  #   negative, neutral, pos = get_sentiment(text[0:500])
  #   all_vals_array = np.array([negative, neutral, pos])
  #   for i in range(500, length, 500):
  #     negative, neutral, pos = get_sentiment(text)
  #     this_vals_array = np.array([negative, neutral, pos])
  #     all_vals_array = np.vstack((all_vals_array, this_vals_array))
  #   means_array = np.mean(all_vals_array, axis=1)
  #   negative, neutral, pos = means_array[0], means_array[1], means_array[2]
  
  chunks = split_text(text)
  negative_ls = []
  neutral_ls = []
  pos_ls = []
  for i in range(len(chunks)):
    # encoded_input = tokenizer(chunks[i], return_tensors='pt')
    # output = model(**encoded_input)
    # scores = output[0][0].detach().numpy()
    # scores = softmax(scores)

    # negative, neutral, pos = scores[0], scores[1], scores[2]
    # print(len)
    negative, neutral, pos = get_sentiment(chunks[i])
    negative_ls.append(negative)
    neutral_ls.append(neutral)
    pos_ls.append(pos)


  negative = np.mean(negative_ls)
  neutral = np.mean(neutral_ls)
  pos = np.mean(pos_ls)


  




  file_name_save = clean_file_name+case_name+".txt"
  file_name_save = file_name.replace(" ", "")
  # else:
  #   negative, neutral, pos = get_sentiment(text)
  non_stop_words_count = remove_stop_words(text)
      
  new_row = [case_name, ceqa, name_ls, names_count, parking_count, affordable_count, community_count, words_count, non_stop_words_count, pos, negative, neutral, date, area, file_name_save]
  #df = df.append(pd.Series(new_row, index=df.columns), ignore_index=True)
  df.loc[len(df)] = new_row
  df.to_csv('cases.tsv', sep='\t',index=False)



txt/01-05-10audio_01DepartmentalReport.txt


txt/01-05-10audio_02CommissionBusiness.txt
txt/01-05-10audio_03ZA09-2885_04ZA09-2886b.txt
txt/01-05-10audio_04ZA09-2886.txt
txt/01-05-10audio_05APCW10-657.txt
txt/01-05-10audio_06APCW10-2130.txt
txt/01-05-10audio_07ZA07-5584b.txt
txt/01-09-13audio_01DepartmentalReport.txt
txt/01-09-13audio_02CommissionBusiness.txt
txt/01-09-13audio_03APCE12-1723b.txt
txt/01-09-13audio_04PublicCommentPeriod.txt
txt/01-10-2017audio_1 DirRpt.txt
txt/01-10-2017audio_2 CommBus.txt
txt/01-10-2017audio_3 NCP.txt
txt/01-10-2017audio_4 VTT-72513.txt
txt/01-10-2017audio_5 PubCom.txt
txt/01-11-2011audio_01DirectorsReport.txt
txt/01-11-2011audio_02CommissionBusiness.txt
txt/01-11-2011audio_03ZA08-3851.txt
txt/01-11-2011audio_04ZA09-4164b.txt
txt/01-11-2011audio_05ZA09-2116.txt
txt/01-11-2011audio_06ZA09-1185.txt
txt/01-11-2011audio_07DIR10-1338b.txt
txt/01-11-2017audio_1 DirRpt.txt
txt/01-11-2017audio_2 CommBus.txt
txt/01-11-2017audio_3 NCP.txt
txt/01-11-2017audio_4 ZA-2015-1109.txt
txt/01-11-2017audio_5 ZA-2016-1

KeyboardInterrupt: 

In [ ]:
df

,case_name,CEQA,names,names_count,parking_mentions,affordability_mentions,community_mentions,num_words,num_nonstopgap_words,sentiment_pos,sentiment_neg,sentiment_neutral,date,area,txt_file_name
0,1 DirRp,NaN,[],0,0,0,0,305,178,0.433737,0.023680,0.542582,1/25/17,East,txt/01-25-2017audio_1DirRpt.txt
1,2 CommBu,NaN,[],0,0,0,0,342,187,0.195975,0.032334,0.771691,1/25/17,East,txt/01-25-2017audio_2CommBus.txt
2,3 NC,NaN,[],0,0,0,0,39,21,0.180984,0.064153,0.754864,1/25/17,East,txt/01-25-2017audio_3NCP.txt
3,4 VTT-74301+PubCo,ENV-2016-1898-CE,[],0,0,0,0,0,0,0.270926,0.058704,0.670370,1/25/17,East,txt/01-25-2017audio_4VTT-74301+PubCom.txt
4,01DirectorsRepor,NaN,"[(Bill, Rochen)]",1,0,0,3,2739,1356,0.341688,0.038798,0.619515,7/12/12,CPC,txt/07-12-2012audio_01DirectorsReport.txt
5,02CommissionBusines,NaN,[],0,5,0,1,4270,2106,0.337581,0.077420,0.584999,7/12/12,CPC,txt/07-12-2012audio_02CommissionBusiness.txt
6,03PublicCommentPerio,NaN,"[(Armando, Herman)]",1,0,0,0,390,208,0.445862,0.013924,0.540214,7/12/12,CPC,txt/07-12-2012audio_03PublicCommentPeriod.txt
7,04CPC08-2142,NaN,"[(Rigoberto, Urtado), (Anna, C), (Isabel, Roja...",24,0,0,1,28822,14536,0.194725,0.166172,0.639103,7/12/12,CPC,txt/07-12-2012audio_04CPC08-2142f.txt
8,05CPC12-1504,ENV-2012-1505-CE,"[(George, U), (King, Chung), (Tari, Statelyn),...",20,0,2,7,16639,8698,0.233360,0.172151,0.594489,7/12/12,CPC,txt/07-12-2012audio_05CPC12-1504c.txt
9,1 DepRp,NaN,"[(Harden, Carter)]",1,0,1,0,1402,714,0.296306,0.028725,0.674969,7/20/16,West,txt/WLA07-20-16audio_1DepRpt.txt


In [ ]:
collapsed_df

,clean_file_name,case_name,audio_file_name
0,01-25-2017audio,1 DirRp,[01-25-2017audio_1 DirRpt]
1,01-25-2017audio,2 CommBu,[01-25-2017audio_2 CommBus]
2,01-25-2017audio,3 NC,[01-25-2017audio_3 NCP]
3,01-25-2017audio,4 VTT-74301+PubCo,[01-25-2017audio_4 VTT-74301+PubCom]
4,07-12-2012audio,01DirectorsRepor,[07-12-2012audio_01DirectorsReport]
5,07-12-2012audio,02CommissionBusines,[07-12-2012audio_02CommissionBusiness]
6,07-12-2012audio,03PublicCommentPerio,[07-12-2012audio_03PublicCommentPeriod]
7,07-12-2012audio,04CPC08-2142,"[07-12-2012audio_04CPC08-2142a, 07-12-2012audi..."
8,07-12-2012audio,05CPC12-1504,"[07-12-2012audio_05CPC12-1504a, 07-12-2012audi..."
9,WLA 07-20-16audio,1 DepRp,[WLA 07-20-16audio_1 DepRpt]
